In [1]:
import pandas as pd


In [2]:
df_tennis=pd.read_csv("Prog 3-data3.csv")
print("\n given play tennis dataset:\n\n",df_tennis)



 given play tennis dataset:

      Outlook Temperature Humidity    Wind Answer
0      sunny         hot     high    weak     no
1      sunny         hot     high  strong     no
2   overcast         hot     high    weak    yes
3       rain        mild     high    weak    yes
4       rain        cool   normal    weak    yes
5       rain        cool   normal  strong     no
6   overcast        cool   normal  strong    yes
7      sunny        mild     high    weak     no
8      sunny        cool   normal    weak    yes
9       rain        mild   normal    weak    yes
10     sunny        mild   normal  strong    yes
11  overcast        mild     high  strong    yes
12  overcast         hot   normal    weak    yes
13      rain        mild     high  strong     no


In [3]:
def entropy(probs):
    import math
    return sum([-prob*math.log(prob,2)for prob in probs])

In [4]:
def entropy_of_list(a_list):
    from collections import Counter
    cnt=Counter(x for x in a_list)
    num_instances=len(a_list)
    probs=[x/num_instances for x in cnt.values()]
    return entropy(probs)

In [5]:
total_entropy=entropy_of_list(df_tennis['Answer'])
print("\n total entropy of play tennis dataset:",total_entropy)



 total entropy of play tennis dataset: 0.9402859586706309


In [6]:
def information_gain(df,split_attribute_name,target_attribute_name,trace=0):
    df_split=df.groupby(split_attribute_name)
    for name,group in df_split:
        nobs=len(df.index)
    df_agg_cnt=df_split.agg({target_attribute_name:[entropy_of_list,lambda x:len(x)/nobs]})[target_attribute_name]
    df_agg_cnt.columns=['entropy','propobservations']
    if trace:
        print(df_agg_cnt)
    new_entropy=sum(df_agg_cnt['entropy']*df_agg_cnt['propobservations'])
    old_entropy=entropy_of_list(df[target_attribute_name])
    return old_entropy-new_entropy
print('info-gain for outlook is:'+str(information_gain(df_tennis,'Outlook','Answer')),"\n")
print('info-gain for humidity is:'+str(information_gain(df_tennis,'Humidity','Answer')),"\n")
print('info-gain for wind is:'+str(information_gain(df_tennis,'Wind','Answer')),"\n")
print('info-gain for temperature is:'+str(information_gain(df_tennis,'Temperature','Answer')),"\n")




info-gain for outlook is:0.2467498197744391 

info-gain for humidity is:0.15183550136234136 

info-gain for wind is:0.04812703040826927 

info-gain for temperature is:0.029222565658954647 



In [7]:
def id3(df,target_attribute_name,attribute_names,default_class=None):
    from collections import Counter
    cnt=Counter(x for x in df[target_attribute_name])
    if len(cnt)==1:
        return next(iter(cnt))
    elif df.empty or (not attribute_names):
        return default_class
    else:
        defualt_class=max(cnt.keys())
    gainz=[information_gain(df,attr,target_attribute_name) for attr in attribute_names]
    index_of_max=gainz.index(max(gainz))
    best_attr=attribute_names[index_of_max]
    tree={best_attr:{}}
    remaining_attribute_names=[i for i in attribute_names if i!=best_attr]
    for attr_val,data_subset in df.groupby(best_attr):
        subtree=id3(data_subset,target_attribute_name,remaining_attribute_names,default_class)
        tree[best_attr][attr_val] =subtree
    return tree    

In [8]:
attribute_names=list(df_tennis.columns)
attribute_names.remove('Answer')
from pprint import pprint
tree=id3(df_tennis,'Answer',attribute_names)
print("\n\n The resultant decision tree is:\n")
pprint(tree)



 The resultant decision tree is:

{'Outlook': {'overcast': 'yes',
             'rain': {'Wind': {'strong': 'no', 'weak': 'yes'}},
             'sunny': {'Humidity': {'high': 'no', 'normal': 'yes'}}}}


In [9]:
def predict(query,tree,default=1):
    for key in list(query.keys()):
        if key in list(tree.keys()):
            #try:
            result=tree[key][query[key]]
           # except:
               # return default
            result=tree[key][query[key]]
            if isinstance(result,dict):
                return predict(query,result)
            else:
                return result
query={'Outlook':'sunny','Temperature':'cool','Humidity':'high','Wind':'weak'}
answer=predict(query,tree)
print('\n Can tennis be played for given sample:',answer)


 Can tennis be played for given sample: no
